In [1]:
import requests
import os
import base64
import csv
import json
import re
import datetime
from spotify_client import *
from credentials.config import *

In [2]:
def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())
    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }
    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': f'{refresh_token}'    
    }
    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)
    refresh_response = r_token.json()
    access_token = refresh_response['access_token']
    return access_token


playlist_token = refresh_accesss_token()
csv_path = os.path.join('..', 'data', 'meta_scrape.csv')

In [3]:
###
# imported get_date_info into tkinter as a default entry in for the scrape button
# left here in case using the scrape without the gui
###
def get_date_info():    
    my_date = datetime.date.today() 
    year, week_num, day_of_week = my_date.isocalendar()
    date_info = (year, week_num)
    return date_info

In [4]:
def get_track_features(track_uris):

    playlist_token = refresh_accesss_token()
    track_ids = [track[14:] for track in track_uris]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
#     url = f'https://api.spotify.com/v1/audio-features?ids='
#     =7ouMYWpwJ422jRcDASZB7P%2C4VqPOruhp5EdPBeR92t6lQ%2C2takcwOaAZWiXQijPHIx7B'
    for batch in batched_tracks:
        request_data =  "%2C".join(batch)
        
        url = f'https://api.spotify.com/v1/audio-features?ids={request_data}'
        response = requests.get(
            url,
            headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
 
        b = response.json()

In [5]:
def get_album_tracks(album_ids):
    track_uris = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
        tracks = response_json['items']
        for track in tracks:
            track_uris.append(track['uri'])
    return track_uris

In [6]:
def search_for_albums(week_num, csv_path):

    artists=[]
    albums=[]
    meta_score=[]
    user_score=[]

    albums_not_found = {'artist': [], 'album': []}
    album_ids = set()
    

#     read in weekly metaScrape csv 

    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            
#         filter for artists and albums from current week
            if int(row['week_num'])== week_num:

                artists.append(row['artist'])
                albums.append(row['album'])
                meta_score.append(row['meta_score'])
                user_score.append(row['user_score'])
#   initialize spotify client
 
    spotify = SpotifyAPI(client_id, client_secret)
    for al, ar in zip(albums, artists):
#       trim album names of suffixes (e.g. .vol 1, [ep] etc.)
        al_trim = re.split(',|\[', al)
#       search for album ids 
        temp = spotify.search({"album":al_trim[0]}, search_type="album")
        try:
            parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
        except:
            albums_not_found['artist'].append(ar)
            albums_not_found['album'].append(al) 
        album_ids.add(parse_album_ids)
#   create log of albums_not_found
    return(get_album_tracks(album_ids))

In [7]:
def add_tracks_to_playlist(week_num, playlist_id):
    track_uris = [track for track in search_for_albums(week_num, csv_path)]
 
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_uris[i * track_limit:(i + 1) * track_limit] for i in range((len(track_uris) + track_limit - 1) // track_limit)]  
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    for batch in batched_tracks:
        request_data = json.dumps(batch)

        requests.post(
            url,
            data=request_data,
            headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
    return

In [8]:
def create_playlist(week_num):
    date_info =  get_date_info()
    year = date_info[0]
    
    request_body = json.dumps({
        'name': f'{year}-week {week_num} scrape',
        'description': f'metacritic rated albums for the {week_num}th of the year',
        'public': True
    })
    url = f'https://api.spotify.com/v1/users/{spotify_user_id}/playlists'


    response = requests.post(
        url,
        data = request_body,
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'

    })
    response_json = response.json()
    playlist_id = (response_json['id'])
    # code here for traking playlists by writing playlist ids to file
    add_tracks_to_playlist(week_num, playlist_id)

In [9]:
import pandas as pd
import glob

In [10]:
folder = '../data/hist_scrape/'
path = folder+'albums_*.csv'
csv_paths = glob.glob(path, recursive=True)

dfs = [pd.read_csv(csv_path) for csv_path in csv_paths]
df = pd.concat(dfs)

In [15]:
a = 10
print(a)

10


In [17]:
df

,date,year,week_num,album,artist,meta_score,user_score,crit_num,user_num,label,genre
0,"October 31, 2000",2000,44,Stankonia,Outkast,95,88,20,447,Arista/La Face,Rap
1,"May 2, 2000",2000,18,XTRMNTR,Primal Scream,90,88,16,58,Astralwerks,Alternative
2,"May 2, 2000",2000,18,Bachelor No 2 or the last remains of the dodo,Aimee Mann,89,86,13,34,Superego,Alternative
3,"October 24, 2000",2000,43,Stories from the City Stories from the Sea,PJ Harvey,88,89,23,115,Island,Alternative
4,"November 21, 2000",2000,47,Mass Romantic,The New Pornographers,87,88,8,42,Mint Records,Pop
...,...,...,...,...,...,...,...,...,...,...,...
464,"June 19, 2020",2020,25,Bigger Love,John Legend,59,71,7,15,Columbia,Soul
465,"January 10, 2020",2020,2,The Deadbeat Bang of Heartbreak City,Beach Slang,59,45,9,17,Bridge Nine Records,Alternative Pop/Rock
466,"August 28, 2020",2020,35,Smile,Katy Perry,58,83,18,11466,Capitol,Pop
467,"February 14, 2020",2020,7,Changes,Justin Bieber,57,53,16,4342,Def Jam,Pop


In [12]:
dir(df)

['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__re

In [66]:
na_df = df[df['label'].isna()]
na_df

,date,year,week_num,album,artist,meta_score,user_score,crit_num,user_num,label,genre
61,"September 19, 2000",2000,38,SelmaSongs Music from the Motion Picture Dance...,Bjrk,76,85,NaN,0,NaN,unknown
77,"June 20, 2000",2000,25,3,Pole,73,0,NaN,0,NaN,unknown
111,"October 17, 2000",2000,42,Eat at Whiteys,Everlast,67,84,NaN,0,NaN,unknown
6,"August 28, 2001",2001,35,Vespertine,Bjrk,88,90,NaN,0,NaN,unknown
12,"May 15, 2001",2001,20,Ágtis Byrjun,Sigur Rós,87,89,NaN,0,NaN,unknown
...,...,...,...,...,...,...,...,...,...,...,...
338,"January 31, 2020",2020,5,ACCA,AlaNi,75,0,7,0,NaN,Vocal Jazz
374,"April 24, 2020",2020,17,Harkin,Harkin,73,0,8,0,NaN,Pop/Rock
412,"October 2, 2020",2020,40,The Album,BlackPink,71,85,NaN,0,NaN,unknown
417,"February 21, 2020",2020,8,Names of North End Women,Lee RanaldoRal Refree,70,0,NaN,0,NaN,unknown


In [68]:
us_df = df.loc[df.artist=='Holly Golightly the Brokeoffs']
us_df

,date,year,week_num,album,artist,meta_score,user_score,crit_num,user_num,label,genre
343,"March 30, 2010",2010,13,Medicine County,Holly Golightly the Brokeoffs,73,0,NaN,0,NaN,unknown
689,"April 26, 2011",2011,17,No Help Coming,Holly Golightly the Brokeoffs,67,0,NaN,0,NaN,unknown


In [54]:
print(na_df['artist'].unique())

['Bjrk' 'Pole' 'Everlast' 'Sigur Rós' 'Ryksopp' 'Sinéad OConnor'
 'The Jon Spencer Blues Explosion' 'All Girl Summer Fun Band' 'Erlend ye'
 'The Ladybug Transistor' 'Devendra Banhart' 'Nas' 'Ministry' 'Beenie Man'
 'Rosebuds' 'Maria Taylor' 'Iron Wine Calexico' 'Maxmo Park' 'Holopaw'
 'Brian Eno David Byrne' '-' 'Barry Adamson' 'B Fleischmann' 'Hard-Fi'
 'Rainer Maria' 'Nelly Furtado' '44' 'Holy Fuck' 'Dizzee Rascal'
 'Lindstrm' 'Wilderness' 'The Notwist' 'Darker My Love' 'Motrhead'
 'Jack Peate' 'Queen Paul Rodgers' 'Sunn O' 'Rodrigo y Gabriela' 'Mew'
 'Florence the Machine' 'Susanna And The Magical Orchestra' 'Dlek'
 'Wooden Birds' 'Discovery' 'Underoath' 'Flying Lotus' 'Gilad Atzmon'
 'The Booksman' 'Retribution Gospel Choir' 'Wrongtom' 'John Legend'
 'Leon Russell' 'Tony Buck' 'Allo Darlin' 'Jaime Meline' 'Dirty Money'
 'Miller' 'Holly Golightly the Brokeoffs' 'The Pariah Dogs' 'Jakob Dylan'
 'Leo Abrahams' 'Ryan Adams the Cardinals' 'Koool G Murder'
 'Planningtorock' 'Patrick Sans

In [49]:
df.loc[df['genre']=='unknown']


,date,year,week_num,album,artist,meta_score,user_score,crit_num,user_num,label,genre
61,"September 19, 2000",2000,38,SelmaSongs Music from the Motion Picture Dance...,Bjrk,76,85,NaN,0,NaN,unknown
77,"June 20, 2000",2000,25,3,Pole,73,0,NaN,0,NaN,unknown
111,"October 17, 2000",2000,42,Eat at Whiteys,Everlast,67,84,NaN,0,NaN,unknown
139,"September 26, 2000",2000,39,Golden Lies,Meat Puppets,60,0,8,0,Breaking Records/Atlantic,unknown
6,"August 28, 2001",2001,35,Vespertine,Bjrk,88,90,NaN,0,NaN,unknown
...,...,...,...,...,...,...,...,...,...,...,...
407,"October 2, 2020",2020,40,Shiver,Jónsi,71,84,16,0,Krunk,unknown
412,"October 2, 2020",2020,40,The Album,BlackPink,71,85,NaN,0,NaN,unknown
417,"February 21, 2020",2020,8,Names of North End Women,Lee RanaldoRal Refree,70,0,NaN,0,NaN,unknown
449,"December 18, 2020",2020,51,Music to Be Murdered By - Side B Deluxe Edition,Eminem,65,89,7,493,Shady Records / Interscope,unknown


In [22]:
clean_df = df.dropna()
clean_df


,date,year,week_num,album,artist,meta_score,user_score,crit_num,user_num,label,genre
0,"October 31, 2000",2000,44,Stankonia,Outkast,95,88,20,447,Arista/La Face,Rap
1,"May 2, 2000",2000,18,XTRMNTR,Primal Scream,90,88,16,58,Astralwerks,Alternative
2,"May 2, 2000",2000,18,Bachelor No 2 or the last remains of the dodo,Aimee Mann,89,86,13,34,Superego,Alternative
3,"October 24, 2000",2000,43,Stories from the City Stories from the Sea,PJ Harvey,88,89,23,115,Island,Alternative
4,"November 21, 2000",2000,47,Mass Romantic,The New Pornographers,87,88,8,42,Mint Records,Pop
...,...,...,...,...,...,...,...,...,...,...,...
464,"June 19, 2020",2020,25,Bigger Love,John Legend,59,71,7,15,Columbia,Soul
465,"January 10, 2020",2020,2,The Deadbeat Bang of Heartbreak City,Beach Slang,59,45,9,17,Bridge Nine Records,Alternative Pop/Rock
466,"August 28, 2020",2020,35,Smile,Katy Perry,58,83,18,11466,Capitol,Pop
467,"February 14, 2020",2020,7,Changes,Justin Bieber,57,53,16,4342,Def Jam,Pop


In [ ]:
# working on getting track features for historical scrape
artists=[]
albums=[]
meta_score=[]
user_score=[]

albums_not_found = {'artist': [], 'album': []}
album_ids = set()


#     read in weekly metaScrape csv 

with open(csv_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:

#         filter for artists and albums from current week
        if int(row['week_num'])== week_num:

            artists.append(row['artist'])
            albums.append(row['album'])
            meta_score.append(row['meta_score'])
            user_score.append(row['user_score'])
#   initialize spotify client

spotify = SpotifyAPI(client_id, client_secret)
for al, ar in zip(albums, artists):
#       trim album names of suffixes (e.g. .vol 1, [ep] etc.)
    al_trim = re.split(',|\[', al)
#       search for album ids 
    temp = spotify.search({"album":al_trim[0]}, search_type="album")
    try:
        parse_album_ids = (temp["albums"]["items"][0]["id"])
#   create dicitonary for albums not found
    except:
        albums_not_found['artist'].append(ar)
        albums_not_found['album'].append(al) 
    album_ids.add(parse_album_ids)
#   create log of albums_not_found
return(get_album_tracks(album_ids))